使用了多種Model來進行資料的預測

最後繳交的版本是參考網路上CNN的Model

# 前置處理

In [ ]:
default_path = '../input/'

In [ ]:
import pandas as pd
train_csv = pd.read_csv(default_path+'train.csv')
test_csv = pd.read_csv(default_path+'test.csv')

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
training_data = np.array(train_csv)
testing_data = np.array(test_csv)
#print(training_data.shape, testing_data.shape)

feature = training_data[:, 1:]
label = training_data[:, :1]

X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size = 0.2, stratify=label)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
import seaborn as sns
sns.set(style='white')
g = sns.countplot(train_csv['label'])

## KNNClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=10)
KNN.fit(X_train, y_train)
print(KNN.predict(X_test[:10]).shape,  y_test[:10].flatten())

print(X_test[:10].shape)

#KNN.score(KNN.predict(X_test[:10]), y_test[:10].flatten())

In [ ]:
prediction = KNN.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, prediction))

In [ ]:
prediction_test = KNN.predict(testing_data)
prediction_test

In [ ]:
result = pd.DataFrame(prediction_test)
result.index += 1
result.index.name = 'ImageId'
result.columns = ['Label']
result.to_csv('results_KNN.csv',header=True)
result

In [ ]:
from google.colab import files
files.download('results_KNN.csv')

## SVC

In [ ]:
from sklearn.svm import SVC
SVC = SVC(kernel = 'poly').fit(X_train, y_train.ravel())
SVC.score(X_test[:1000], y_test[:1000])

In [ ]:
result = pd.DataFrame(SVC.predict(testing_data))
result.index += 1
result.index.name = 'ImageId'
result.columns = ['Label']
result.to_csv('results_SVC.csv',header=True)
result

In [ ]:
from google.colab import files
files.download('results_SVC.csv')

## AdaBoost

In [ ]:
from sklearn import ensemble
AdaBoost = ensemble.AdaBoostClassifier(n_estimators=200, learning_rate=0.1).fit(X_train[:5000], y_train[:5000])
AdaBoost.score(X_test[:1000], y_test[:1000])

## Bagging

In [ ]:
Bagging = ensemble.BaggingClassifier(n_estimators=200).fit(X_train[:5000], y_train[:5000])
Bagging.score(X_test[:1000], y_test[:1000])

## Random Forst

In [ ]:
from sklearn import ensemble
RandomForst = ensemble.RandomForestClassifier(n_estimators=50).fit(X_train, y_train)
RandomForst.score(X_test, y_test)

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB().fit(X_train, y_train)
NB.score(X_test[:1000], y_test[:1000])

## CNN
reference: [Introduction to CNN Keras - Acc 0.997 (top 8%)](https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6)

In [ ]:
# check if any missing value
train_csv.isnull().any().any()

In [ ]:
# Normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Reshape
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28,1)

In [ ]:
# One-hot encoding
from keras.utils import to_categorical
y_train_oneHot = to_categorical(y_train, num_classes= 10)
y_test_oneHot = to_categorical(y_test, num_classes=10)

In [ ]:
#X_train[0][:, :, 0]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[0][:, :, 0])

In [ ]:
from keras.layers import Convolution2D, Dense, MaxPooling2D, Dropout, Flatten
from keras.models import Sequential

model = Sequential()
model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same', input_shape = (28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(filters=64, kernel_size=(5, 5), padding='same', activation='relu'))       
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(units=256, activation='relu',))
model.add(Dropout(0.3))
model.add(Dense(units=10, activation='softmax'))

model.summary()

### Different optimizer
Choose one to execute

In [ ]:
# optimizer = 'adam'
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# optimizer = 'RMSprop'
model.compile(optimizer='RMSprop', loss = 'categorical_crossentropy', metrics=['accuracy'])

### .

In [ ]:
from keras.callbacks import ReduceLROnPlateau
# To keep the advantage of the fast computation time with a high LR, i decreased the LR dynamically every X steps (epochs) depending if it is necessary (when accuracy is not improved).
# With the ReduceLROnPlateau function from Keras.callbacks, i choose to reduce the LR by half if the accuracy is not improved after 3 epochs.

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
#model.fit(X_train, y_train_oneHot, batch_size=256, epochs=10,verbose=1, validation_split=0.2, callbacks=[learning_rate_reduction])
model.fit(X_train, y_train_oneHot, batch_size=256, epochs=10,verbose=1, validation_split=0.2)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# I did not apply a vertical_flip nor horizontal_flip since it could have lead to misclassify symetrical numbers such as 6 and 9.

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(X_train)



In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(X_train, y_train_oneHot, batch_size=128),
                              epochs = 50, validation_data=(X_test, y_test_oneHot),
                              verbose = 1, steps_per_epoch=X_train.shape[0]/128, 
                              callbacks=[learning_rate_reduction])

In [ ]:
model.evaluate(X_test, y_test_oneHot)

In [ ]:
testing_data = scaler.transform(testing_data).reshape(-1, 28, 28, 1)
CNN_prediction = model.predict_classes(testing_data)

In [ ]:
result = pd.DataFrame(CNN_prediction)
result.index += 1
result.index.name = 'ImageId'
result.columns = ['Label']
result.to_csv('results_CNN_.csv',header=True)
result.head()